In [1]:
## third-party
from transformers import AutoTokenizer
import torch
import sys
sys.path.append('./')
## own
from xRAG.src.model import SFR,XMistralForCausalLM
from xRAG.src.language_modeling.utils import get_retrieval_embeds,XRAG_TOKEN

/home/hdejean/miniconda3/envs/bergen12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0")
llm_name_or_path = "Hannibal046/xrag-7b"
llm = XMistralForCausalLM.from_pretrained(llm_name_or_path,torch_dtype = torch.bfloat16,low_cpu_mem_usage = True,).eval()
llm_tokenizer = AutoTokenizer.from_pretrained(llm_name_or_path,add_eos_token=False,use_fast=False,padding_side='left')

## here, XRAG_TOKEN is just a place holder
llm.set_xrag_token_id(llm_tokenizer.convert_tokens_to_ids(XRAG_TOKEN))
print(XRAG_TOKEN)

/home/hdejean/miniconda3/envs/bergen12/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.36it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<xRAG>


In [5]:
retriever_name_or_path = "Salesforce/SFR-Embedding-Mistral"
retriever = SFR.from_pretrained(retriever_name_or_path,torch_dtype = torch.bfloat16).eval()
retriever_tokenizer = AutoTokenizer.from_pretrained(retriever_name_or_path)

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.97s/it]


In [3]:
question = """What company advertised itself with the slogan "We'll leave a light on for you"?"""
documents = [
    'Document 1: Alvin and the Chipmunks | " Alvin and the Chipmunks, originally David Seville and the Chipmunks or simply The Chipmunks, are an American animated virtual band created by Ross Bagdasarian for a novelty record in 1958. The group consists of three singing animated anthropomorphic chipmunks named Alvin, Simon, and Theodore. They are managed by their human adoptive father, David ""Dave"" Seville. Bagdasarian provided the group\'s voices sped up to create high-pitched squeaky voices (which wasn\'t entirely new to him, having worked on ""Witch Doctor"" earned the record two Grammy Awards for engineering). ""The Chipmunk Song"" became a number-one single in the United States. After Bagdasarian died in 1972, the characters’ voices were provided by his son Ross Bagdasarian Jr. and the latter\'s wife Janice Karman in the subsequent incarnations of "',
    "Document 2: Jamie Lee Curtis |  Jamie Lee Curtis (born November 22, 1958) is an American actress and writer. She is the recipient of several accolades, including a British Academy Film Award, two Golden Globe Awards and a star on the Hollywood Walk of Fame in 1998. Curtis made her film acting debut as Laurie Strode in John Carpenter's horror film Halloween (1978), which established her as a scream queen, and she thereafter appeared in a string of horror films, including The Fog, Prom Night, Terror Train (all 1980) and Roadgames (1981). She reprised the role of Laurie in the sequels Halloween II (1981), Halloween H20: 20 Years Later (1998), Halloween: Resurrection (2002), Halloween (2018), and Halloween Kills (2021). Her filmography is largely characterized by independent film that have been box-office successes, with 8 of her lead-actress credits ",
    'Document 3: Sunset Boulevard (musical) | " The American premiere was at the Shubert Theatre in Century City, Los Angeles, California, on 9 December 1993, with Close as Norma and Alan Campbell as Joe. Featured were George Hearn as Max and Judy Kuhn as Betty. Lloyd Webber had reworked both the book and score, tightening the production, better organising the orchestrations, and adding the song ""Every Movie\'s a Circus"". This new production was better received by the critics and was an instant success, running for 369 performances. The Los Angeles production also recorded a new cast album that is well regarded. It is also the only unabridged cast recording of the show, since the original London recording was trimmed by over thirty minutes. A controversy arose with this production after Faye Dunaway was hired to replace Glenn Close. Dunaway went into rehearsals with Rex Smith as Joe and Jon Cypher as Max. Tickets "',
    'Document 4: Arthur Balfour |  Balfour was appointed prime minister on 12 July 1902 while the King was recovering from his recent appendicitis operation. Changes to the Cabinet were thus not announced until 9 August, when the King was back in London. The new ministers were received in audience and took their oaths on 11 August.',
    'Document 5: Motel 6 | " Beginning in 1986, Motel 6 has advertised through radio commercials featuring the voice of writer and National Public Radio commentator Tom Bodett, with the tagline "We\'ll leave the light on for you." The ads were created by Dallas advertising agency The Richards Group. They feature a tune composed by Tom Faulkner, performed by him on guitar and Milo Deering on fiddle. The first spots were conceived and written by David Fowler. In 1996, the ads won a Clio Award. The campaign itself has won numerous national and international awards and was selected by Advertising Age magazine as one of the Top 100 Advertising Campaigns of the Twentieth Century."',
]

In [32]:
retriever_input = retriever_tokenizer(documents,max_length=180,padding=True,truncation=True,return_tensors='pt')
with torch.no_grad():
    doc_embeds = retriever.get_doc_embedding(input_ids=retriever_input.input_ids,attention_mask=retriever_input.attention_mask)
print(doc_embeds.shape)

torch.Size([5, 4096])


In [33]:
relevant_embedding = doc_embeds[-1]
relevant_embedding.unsqueeze(0)
relevant_embedding.shape


torch.Size([4096])

In [6]:
## 4. concate the doc and query in a template
rag_template = """[INST] 

Background: {document}

Question: {question} [/INST] the answer is """
prompt = rag_template.format_map(dict(question=question,document=XRAG_TOKEN))

template = "[INST] Answer the questions:\n\nQuestion: {question} [/INST] The answer is"
prompt = template.format_map(dict(question=question,document=XRAG_TOKEN))

prompt

'[INST] Answer the questions:\n\nQuestion: What company advertised itself with the slogan "We\'ll leave a light on for you"? [/INST] The answer is'

In [7]:
input_ids = llm_tokenizer(prompt,return_tensors='pt').input_ids
generated_output = llm.generate(
        input_ids = input_ids,
        do_sample=False,
        max_new_tokens=128,
        pad_token_id=llm_tokenizer.pad_token_id,
        #retrieval_embeds = relevant_embedding.unsqueeze(0),
    )
result = llm_tokenizer.batch_decode(generated_output,skip_special_tokens=True)
print(result)

['[INST] Answer the questions:\n\nQuestion: What company advertised itself with the slogan "We\'ll leave a light on for you"? [/INST] The answer is Motel 6. Motel 6 is a budget motel chain in the United States and Canada. The company is known for its slogan "We\'ll leave a light on for you," which signifies a welcoming and friendly atmosphere for weary travelers.']


In [11]:
q="1.\n2"
llm_tokenizer(q,add_special_tokens=False)

{'input_ids': [28705, 28740, 28723, 13, 28750], 'attention_mask': [1, 1, 1, 1, 1]}

In [13]:
llm_tokenizer.decode([28750])

'2'